<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/coviddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima

In [ ]:
!pip install streamlit

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

plt.style.use("seaborn")                  # Establecemos el estilo gráfico de seaborn por defecto
plt.rcParams["figure.figsize"] = (15, 5)  # Tamaño por defecto de las gráficas a crear

In [ ]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/KSchool-Master-Final-Project'       # path to your project on Drive

drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)

In [ ]:
!pwd

In [ ]:
%ls

In [ ]:
%cd My Drive/KSchool-Master-Final-Project

In [ ]:
#Adding git path:

#GIT_USERNAME = "martaarozarena" 
#GIT_TOKEN = "{YOUR_GITHUB_TOKEN}"  
#GIT_REPOSITORY = "KSchool-Master-Final-Project" 

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
#GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
GIT_PATH = 'https://github.com/martaarozarena/KSchool-Master-Final-Project.git'
print("GIT_PATH: ", GIT_PATH)

In [ ]:
!git status

In [ ]:
!git config user.email "marta.arozarenav@gmail.com"
!git config user.name "martaarozarena"

In [ ]:
#!git add DataExploration.ipynb
#!git commit -m "exploring OpenSky data"
#!git push

In [ ]:
!ls

In [ ]:
# covid_url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"  Comento esto, para fijar un fichero mientras desarollamos el codigo
covid=pd.read_csv('data/owid-covid-data 20200713.csv', parse_dates=["date"])    # Dejo fijo el fichero descargado el 13 de julio. 
                                                                           # Podemos añadir index_col ="date" como parámetro dentro del read_csv
covid.info()

In [ ]:
covid[covid['continent']=='Europe'].sample(10)

In [ ]:
covid_eur = covid[covid['continent']=='Europe']
covid_eur.info()

In [ ]:
covid_eur[covid_eur['new_cases'].isnull()]

In [ ]:
covid_eur[covid_eur['new_cases'].isnull()].shape

In [ ]:
covid_eur[covid_eur['location']=='Serbia'].head(15)

In [ ]:
covid_eur.loc[:, ['new_cases']]

In [ ]:
#covid_eur.loc[:, ['new_cases']] = covid_eur['new_cases'].fillna(0)  # The NaN in the 'new_cases' series are supposed to be zero, 
                                                                    # since it was at the beginning when there were still 0 cases in those countries
                                                                    # Warning below?? Do we need to fix it?
covid_eur['new_cases'] = covid_eur['new_cases'].fillna(0)
covid_eur[covid_eur['new_cases'].isnull()].shape                    

In [ ]:
covid_eur

In [ ]:
covid_eur[covid_eur['location'].isin(['United Kingdom'])].plot(x = 'date', y = 'new_cases');  # Keep in mind we need to "fix" the UK series at some point

#From Our World in Data:
#United Kingdom - May 20, July 3:methodologychange
#France - June 2:methodology change
#Spain - April 19 & May25:methodology change
#Italy - June 25:methodology change

In [ ]:
cov_eur_dates = covid_eur[covid_eur['date']<'2020-06-30']    

In [ ]:
plt.figure(figsize = (20,15))
sns.lineplot(x = cov_eur_dates['date'], y = cov_eur_dates['new_cases'], hue = cov_eur_dates['location'], data=cov_eur_dates)

In [ ]:
plt.figure(figsize = (20,15))
sns.lineplot(x = cov_eur_dates['date'], y = cov_eur_dates['total_cases'], hue = cov_eur_dates['location'], data=cov_eur_dates)

In [ ]:
#cogemos España como primer pais para el modelo

In [ ]:
covid_eur[covid_eur['location'].isin(['Spain'])].plot(x = 'date', y = 'new_cases');

In [ ]:
covid_eur[(covid_eur['new_cases']<0) & (covid_eur['location']=="Spain")] #numero de casos negativos 2

In [ ]:
covid_eur[(covid_eur['new_cases']==0) & (covid_eur['location']=="Spain")]#los días con 0 casos tienen sentido, antes de la pandemia y 
#en julio que hubo días de 0 casos

In [ ]:
#seleccionar solo españa y resetear el indice
covid_esp=covid_eur[covid_eur["location"]=="Spain"].reset_index()

In [ ]:
#quitar los negativos si no hacemos moving average
#indices que queremos cambiar porque casos no pueden ser menores de 0
indexes=covid_esp[(covid_esp["new_cases"]<0) &(covid_esp["location"]=="Spain")].index.values
#sustituir por la media de valores anterior y posterior
for i in indexes:
  covid_esp.iloc[i,6]= (covid_esp.iloc[i+1,6]+covid_esp.iloc[i-1,6])/2

In [ ]:
covid_esp.plot(x = 'date', y = 'new_cases');

In [ ]:
covid_esp.describe()

In [ ]:
covid_esp.head()

In [ ]:
covid_esp.index = covid_esp.date

In [ ]:
covid_esp_newcases=covid_esp["new_cases"]

In [ ]:
covid_esp_newcases

In [ ]:
covid_esp_newcases=covid_esp_newcases.resample('D').sum()
covid_esp_newcases

In [ ]:
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import pacf
plot_acf(covid_esp_newcases, unbiased=True)
pyplot.show()
#la autocorrelacion mide como se repite la funcion con el tiempo
#es normal que la autocorrelacion este toda dentro del cono de no confianza porque tenemos muy pocos datos y además la logica dice que 
#tendrian que repetirse anualmente, en primavera mas casos que en verano o en invierno más que en verano etc pero no en un intervalo de 3 meses
plot_pacf(covid_esp_newcases, method="ywm")
pyplot.show()

In [ ]:
#metodo 2, usar moving averages
covid_esp=covid_eur[covid_eur["location"]=="Spain"].reset_index()
covid_esp["MA7"]=covid_esp.new_cases.rolling(7).mean()
covid_esp.index = covid_esp.date
covid_esp_ma7=covid_esp["MA7"]


In [ ]:
covid_esp_ma7

In [ ]:
covid_esp_ma7.plot()

In [ ]:
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plot_acf(covid_esp_ma7.fillna(0))
pyplot.show()
#la autocorrelacion mide como se repite la funcion con el tiempo
#es normal que la autocorrelacion este toda dentro del cono de no confianza porque tenemos muy pocos datos y además la logica dice que 
#tendrian que repetirse anualmente, en primavera mas casos que en verano o en invierno más que en verano etc pero no en un intervalo de 3 meses
plot_pacf(covid_esp_ma7.fillna(0), method="ywm")
pyplot.show()

In [ ]:
import pmdarima as pm
from pmdarima import preprocessing

# Let's look at the series, its ACF plot, and a histogram of its values
pm.tsdisplay(covid_esp_ma7[7:], lag_max=20, title="Spain Covid daily new cases (rolling 7 day average)", show=True)

In [ ]:
covid_esp_ma7[covid_esp_ma7<=0]

In [ ]:
covid_esp_ma7['2020-02-25':].head()

In [ ]:
from scipy.stats import boxcox

covid_esp_ma7_bc, lam = boxcox(covid_esp_ma7['2020-02-25':])
print('Lambda: %f' % lam)

In [ ]:
# The boxcox() function takes an argument, called lambda, that controls the type of transform to perform.
#Below are some common values for lambda

#lambda = -1. is a reciprocal transform.
#lambda = -0.5 is a reciprocal square root transform.
#lambda = 0.0 is a log transform.
#lambda = 0.5 is a square root transform.
#lambda = 1.0 is no transform.

# From https://machinelearningmastery.com/power-transform-time-series-forecast-data-python/ 

In [ ]:
pm.tsdisplay(covid_esp_ma7_bc, lag_max=20, title="Spain Covid daily new cases after boxcox transform", show=True)

In [ ]:
covid_esp_newcases.plot()

In [ ]:
covid_esp_ma7.plot()

In [ ]:
covid_esp_diff = covid_esp_newcases - covid_esp_ma7

In [ ]:
covid_esp_diff.plot()

In [ ]:
pm.tsdisplay(covid_esp_diff[50:], lag_max=20, title="Spain Covid daily new cases after diff transform", show=True)

In [ ]:
# Perform Dicky-Fuller test

# from http://sayef.tech/post/time-series-analysis-in-python-inroduction-to-arima/

from statsmodels.tsa.stattools import adfuller

print('Result of Dicky=Fuller Test')
dftest = adfuller(covid_esp_diff[7:], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', '#Observations Used'])

for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value

print(dfoutput)

In [ ]:
from pmdarima import arima
n_diffs = arima.ndiffs(covid_esp_newcases)
n_diffs

In [ ]:
#el test de dicky-fuller solo analiza tendencia
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    # Determining rolling statistics
    rolling_mean = timeseries.rolling(window=3).mean()
    rolling_std = timeseries.rolling(window=3).std()

    # Plot rolling statistics

    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    std = plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    print('Result of Dicky=Fuller Test')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', '#Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(covid_esp_diff[7:])

In [ ]:
covid_esp_s1 = covid_esp_newcases - covid_esp_newcases.shift(1)

In [ ]:
plt.plot(covid_esp_newcases, color='blue', label='Original')
plt.plot(covid_esp_newcases.shift(4), color='red', label='Shift(4)')
covid_esp_diff4 = covid_esp_newcases - covid_esp_newcases.shift(4)
plt.plot(datadiff, color = 'green', label='Result')
plt.legend(loc='best')
plt.show()

In [ ]:
covid_esp_diff4.head()

In [ ]:
test_stationarity(covid_esp_diff4[4:])

In [ ]:
pm.tsdisplay(covid_esp_diff4[4:], lag_max=20, title="Spain Covid daily new cases after diff transform", show=True)

In [ ]:
test_stationarity(covid_esp_s1[7:])

In [ ]:
covid_esp_logminusma = np.log(covid_esp_newcases[80:]) - covid_esp_ma7[80:]

In [ ]:
covid_esp_logminusma = np.log(covid_esp_newcases['2020-02-25':'2020-06-30']) - covid_esp_newcases['2020-02-25':'2020-06-30']

In [ ]:
test_stationarity(covid_esp_logminusma)

In [ ]:
covid_esp["MA3"]=covid_esp.new_cases.rolling(3).mean()
covid_esp.index = covid_esp.date
covid_esp_ma3=covid_esp["MA3"]

covid_esp_diff3 = covid_esp_newcases - covid_esp_ma3

In [ ]:
test_stationarity(covid_esp_diff3[3:])

In [ ]:
pm.tsdisplay(covid_esp_diff3[3:], lag_max=20, title="Spain Covid daily new cases after diff transform", show=True)

In [ ]:
# https://alkaline-ml.com/pmdarima/auto_examples/index.html#preprocessing-examples

In [ ]:
#Once data is stationary then time to decompose data in trend seasonality and residual 
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(covid_esp_diff3[3:])

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(covid_esp_diff3[3:], label='Original')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')

plt.subplot(413)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='best')

plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()


In [ ]:
decomposed_data = residual
decomposed_data.dropna(inplace=True)
test_stationarity(decomposed_data)

In [ ]:
plot_acf(decomposed_data)

In [ ]:
#to calculate P and q de manera visual
# ACF and PACF plots|:
from statsmodels.tsa.stattools import acf, pacf

lag_acf = acf(covid_esp_diff3[3:], nlags=20)
lag_pacf = pacf(covid_esp_diff3[3:], nlags=20, method='ols')

# Plot ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(covid_esp_diff3[3:])), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(covid_esp_diff3[3:])), linestyle='--', color='gray')
plt.title('ACF')

# Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(covid_esp_diff3[3:])), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(covid_esp_diff3[3:])), linestyle='--', color='gray')
plt.title('PACF')
plt.tight_layout()

In [ ]:
# in both graphs the line cuts for the first time around 2 then p and q are 2

In [ ]:
#forma de buscar los mejores valores de p q y d con un bucle que compara AICs (valor que evalua la calidad del modelo. 0 sería perfecto)
#según esto lo mejor es modelo 4,0,4 es decir p=4 y q=4 
best_aic = 999999
for p in range(5):
    for q in range(5):
        for d in range(3):
            try:
                modelt = SARIMAX(covid_esp_diff3, order = (p, d, q)).fit()
                if modelt.aic < best_aic:
                    best_aic = modelt.aic
                    print(p, d, q, modelt.aic, modelt.bic)
            except:
                pass

In [ ]:
#de manera automatica sale 4,0,4

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

#ARIMAX Model
model1 = SARIMAX(covid_esp_diff3[3:], order=(4, 0, 4))
results_SARIMAX = model1.fit()
plt.plot(covid_esp_diff3[3:])
plt.plot(results_SARIMAX.fittedvalues, color='red')

In [ ]:
results_SARIMAX.summary()

In [ ]:
prediction= results_SARIMAX.get_forecast(steps=12)
prediction.predicted_mean

In [ ]:
prediction.conf_int() #con un 95% de seguridad el valor de 13 de julio está entre -866 y 756 ........

In [ ]:
covid_esp_diff3[3:].plot()
prediction.predicted_mean.plot()
plt.fill_between(prediction.predicted_mean.index,prediction.conf_int()["lower y"],prediction.conf_int()["upper y"])
plt.show()

In [ ]:
results_SARIMAX.fittedvalues

In [ ]:
covid_esp_diff3[3:].plot()
results_SARIMAX.fittedvalues.plot()

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(covid_esp_diff3[3:],results_SARIMAX.fittedvalues)

In [ ]:
results_SARIMAX.plot_diagnostics()
plt.show();

In [ ]:
##Prueba con la funcion automatica de pmdarima. sale un q=1 es decir modelo AM puro
model2=pm.auto_arima(covid_esp_diff3[3:],seasonal=False , suppress_warnings=True)
model2.summary()

In [ ]:
print(model2.order)

In [ ]:
model2.plot_diagnostics()

In [ ]:
#Ahora habría que deshacer el cambio
#covid_esp_diff3 = covid_esp_newcases - covid_esp_ma3
#covid_esp_newcases= covid_esp_diff3+covid_esp_ma3

In [ ]:
#covid_esp["MA3"]=covid_esp.new_cases.rolling(3).mean()
#covid_esp.index = covid_esp.date
#covid_esp_ma3=covid_esp["MA3"]

#covid_esp_diff3 = covid_esp_newcases - covid_esp_ma3

from statsmodels.tsa.arima_model import ARIMA

model3 = ARIMA(covid_esp_newcases, order=(4, 0, 4))
results_ARIMA = model3.fit()
plt.plot(covid_esp_newcases)
plt.plot(results_ARIMA.fittedvalues, color='red')


In [ ]:
prediction3 = results_ARIMA.forecast(steps=12)
prediction3

In [ ]:
covid_esp_newcases.shape

In [ ]:
prediction3 = results_ARIMA.forecast(steps=12)

results_ARIMA.plot_predict(1,205)

In [ ]:
#probamos con 1 variable exogena
#ARIMAX Model
flights_esp = pd.read_csv("vuelos_esp.csv",parse_dates=["FLT_DATE"],index_col="FLT_DATE")
flights_esp=flights_esp["FLT_ARR_1"]
flights_esp=flights_esp.resample("1D").sum()
flights_esp
#solo hay vuelos hasta 31-5-2020 y el modelo covid va hasta 12-7-2020

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model4 = SARIMAX(covid_esp_diff3[3:-12], order=(4, 0, 4),exog=flights_esp[2:])
results_SARIMAX = model4.fit()
plt.plot(covid_esp_diff3[3:-12])
plt.plot(results_SARIMAX.fittedvalues, color='red')

In [ ]:
prediction4= results_SARIMAX.get_forecast(steps=12,exog=[688,553,616,630,612,688,553,616,630,612,688,533])# poner por dia de la semana

In [ ]:
covid_esp_diff3[3:-12].plot()
prediction4.predicted_mean.plot()
plt.fill_between(prediction4.predicted_mean.index,prediction4.conf_int()["lower y"],prediction4.conf_int()["upper y"])
plt.show()

In [ ]:
covid_esp.new_cases[1:-12]

In [ ]:
prediction4.predicted_mean

In [ ]:
(200+301+x)/3=146.52=-61,55

In [ ]:
#covid_esp_diff3 = covid_esp_newcases - covid_esp_ma3
# 146.53=x-(200+301+x/)3
covid_esp_bucle=covid_esp.new_cases.copy()
covid_esp_bucle=covid_esp_bucle[:-12]
fecha=covid_esp_bucle.index[-1]
j=1
covid_esp_bucle
  


In [ ]:

for i in  prediction4.predicted_mean:
  inicial_2=covid_esp_bucle[-2]
  inicial_1=covid_esp_bucle[-1]
  x=(-inicial_2-inicial_1-(3*i))/-2
  covid_esp_bucle.append(pd.Series(x,index=fecha,))
  j=j+1

In [ ]:
plt.plot(covid_esp_bucle[-15:],color="blue")
plt.plot(covid_esp.new_cases[-15:],color="red")

In [ ]:
covid_esp.new_cases[-15:]

In [ ]:
covid_esp_bucle[-15:]

In [ ]:
### Creating a front end visualization

In [ ]:
pip install tornado==5.0

In [1]:
import streamlit

In [ ]:
##ngrok is the name of the library we need if we want to create a public url for colab since colab doesnt allow to use the urls
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



In [6]:
streamlit hello #this doesnt work here it needs to be used in the terminal. 
#in streamlit you create the file .py and then you run it from terminal not from the notebook

SyntaxError: invalid syntax (<ipython-input-6-c7a1b683aa76>, line 1)

In [ ]:
st.title("Coronavirus deaths forecast ")
st.markdown("explain text of what this visualization makes")